# CIS 5300 Fall 2022 - Homework 4 | Semantic Parsing with Encoder-Decoder Models

#Introduction

In this homework, you are going to implement a Seq2Seq model for semantic parsing. We will be converting natural language sentences into queries we can execute against a knowledgebase. To start you off, we've provided you code that set up your environment. First, run a nearest neighbor baseline that we provided to understand how the process looks like. Then you will need to implement your own Seq2Seq model and evalute on it.

You should NOT change the class names, as parts of the code we are providing to simplify the developement depends on it. For each class, some initalization of parameters and the starter code are alreay provided, so you can simply fill in the code between `### BEGIN OF FILL-IN ###` and `### END OF FILL-IN ###` to complete this assignment. However, feel free to change the starter code and even the initalization of parameters if you prefer this way. In the end, you will only be evaluated on the quality of your outputs: as long as your model reaches the target performance levels, using Seq2Seq models, you will get full credit, regardless of implementation details.

# Preperation

You should select the device (CPU vs GPU) in Colab. The whole homework is runable by CPU within a reasonable time (about 8 minutes), but GPU can speed up the model implementation more (about 2 minutes).

Here, we provided the command line code to download the starter code from GitHub repo. You may also want to go through some other .py files in this project as some of the functions and classes are imported and used here, especially the *evaluate()* function from *lf_evaluator.py* and *Example()*, *Derivation()*, *load_datasets()* and some other functions from *data.py*. After you run the cell to clone the repo, you will find the files on the left-hand side in colab under Files, or https://github.com/realliyifei/cis5300-semantic-parsing.

#Setting up

The following few sections will help you set up java (needed for eval) and some other helper code and packages that may help you throughout this assignment. Please do not change them and run them first.

In [1]:
#@title Set-up Java
import os       #importing os to set environment variable
def install_java():
#   !apt update
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "1.8.0_412"
OpenJDK Runtime Environment (build 1.8.0_412-8u412-ga-1~22.04.1-b08)
OpenJDK 64-Bit Server VM (build 25.412-b08, mixed mode)


In [2]:
# !git clone https://github.com/realliyifei/cis5300-semantic-parsing
# %cd cis5300-semantic-parsing
# !chmod a+x evaluator/geoquery # Solve permission issue on the `geoquery` file

In [3]:
import argparse
import random
import numpy as np
from typing import List
from tqdm.notebook import tqdm
# import times

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable as Var
from torch import optim

# Import from starter code
from utils import *
from data import *
from lf_evaluator import *

## Argument Parser

Here we start you off with some common arguments that you may need for this homework. Feel free to extend it and include whatever arguments you feel needed for your model later!

In [4]:
def add_models_args(parser):
    """
    Command-line arguments to the system related to your model.  Feel free to extend here.  
    """
    # Set device
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    parser.add_argument('--device', type=str, default=device, help='set device')

    # Some common arguments for your convenience
    parser.add_argument('--seed', type=int, default=0, help='RNG seed (default = 0)')
    parser.add_argument('--epochs', type=int, default=10, help='num epochs to train for')
    parser.add_argument('--lr', type=float, default=1e-3)
    parser.add_argument('--batch_size', type=int, default=2, help='batch size')

    # 65 is all you need for GeoQuery
    parser.add_argument('--decoder_len_limit', type=int, default=65, help='output length limit of the decoder')

    ### BEGIN OF FILL-IN ###
    # Feel free to add other hyperparameters for your input dimension, etc. to control your network
    # 50-200 might be a good range to start with for embedding and LSTM sizes
    parser.add_argument('--reverse_input', action='store_true', help='reverse the input sequence')
    parser.add_argument('--emb_dim', type=int, default=100, help='size of word embeddings')
    parser.add_argument('--hidden_size', type=int, default=200, help='size of LSTM hidden states')
    parser.add_argument('--attention', action='store_true', help='use attention mechanism')
    parser.add_argument('--attention_type', type=str, default='additive', choices=['additive', 'multiplicative'], help='type of attention mechanism')
    parser.add_argument('--output_max_len', type=int, default=65, help='maximum length of the output sequence')
    parser.add_argument('--rnn_type', type=str, default='lstm', choices=['lstm', 'gru'], help='type of RNN to use')
    parser.add_argument('--dropout', type=float, default=0.2, help='dropout probability')

    ### END OF FILL-IN ###


# Baseline

## Nearest Neighbor Baseline

This is a baseline implementation of a nearest neighbor model that we provided for you. It returns the query of the training string that has the highest token overlap with a test string. We also provided you some code for you to run this model. You can run the following sections here and see how well it does to verify your setup and get some insights of how to run and evaluate your own model later on. It should get about 24/120 denotation accuracy.

##Nearest Neighbor Model Implementation


In [5]:
class NearestNeighborSemanticParser(object):
    """
    Semantic parser that uses Jaccard similarity to find the most similar input example to a particular question and
    returns the associated logical form.
    """
    def __init__(self, training_data: List[Example]):
        self.training_data = training_data

    def decode(self, test_data: List[Example]) -> List[List[Derivation]]:
        """
        :param test_data: List[Example] to decode
        :return: A list of k-best lists of Derivations. A Derivation consists of the underlying Example, a probability,
        and a tokenized input string. If you're just doing one-best decoding of example ex and you
        produce output y_tok, you can just return the k-best list [Derivation(ex, 1.0, y_tok)]
        """
        test_derivs = []
        for test_ex in test_data:
            test_words = test_ex.x_tok
            best_jaccard = -1
            best_train_ex = None
            # Find the highest word overlap with the train data
            for train_ex in self.training_data:
                # Compute word overlap with Jaccard similarity
                train_words = train_ex.x_tok
                overlap = len(frozenset(train_words) & frozenset(test_words))
                jaccard = overlap/float(len(frozenset(train_words) | frozenset(test_words)))
                if jaccard > best_jaccard:
                    best_jaccard = jaccard
                    best_train_ex = train_ex
            # Note that this is a list of a single Derivation
            test_derivs.append([Derivation(test_ex, 1.0, best_train_ex.y_tok)])
        return test_derivs

##Nearest Neighbor Runner Code

The following section includes code that help you run the nearest neighbor model. This could be a good template for you to run your model as well. Notice that here the _parse_args() function calls the add_models_args() above. So you can extend the add_models_args() function as before to include any arguments you may want. Another thing to notice here is that we set the argument do_nearest_neighbor=True to run this nearest neighbor experiment, and you may want to set it to False to run your own model.

In [6]:
def _parse_args():
    """
    Command-line arguments to the system. --model switches between the main modes you'll need to use. The other arguments
    are provided for convenience.
    :return: the parsed args bundle
    """
    parser = argparse.ArgumentParser(description='main.py')
    
    # General system running and configuration options
    parser.add_argument('--do_nearest_neighbor', dest='do_nearest_neighbor', default=False, action='store_true', help='run the nearest neighbor model')

    parser.add_argument('--train_path', type=str, default='data/geo_train.tsv', help='path to train data')
    parser.add_argument('--dev_path', type=str, default='data/geo_dev.tsv', help='path to dev data')
    parser.add_argument('--test_path', type=str, default='data/geo_test.tsv', help='path to blind test data')
    parser.add_argument('--test_output_path', type=str, default='test_output.tsv', help='path to write blind test results')
    parser.add_argument('--domain', type=str, default='geo', help='domain (geo for geoquery)')
    parser.add_argument('--no_java_eval', dest='perform_java_eval', default=True, action='store_false', help='run evaluation of constructed query against java backend')
    parser.add_argument('--print_dataset', dest='print_dataset', default=False, action='store_true', help="Print some sample data on loading")
    add_models_args(parser) 

    args = parser.parse_args("")
    return args

def main(args):
    # Load the training and test data 
    train, dev, test = load_datasets(args.train_path, args.dev_path, args.test_path, domain=args.domain)
    train_data_indexed, dev_data_indexed, test_data_indexed, input_indexer, output_indexer = index_datasets(train, dev, test, args.decoder_len_limit)
    print("%i train exs, %i dev exs, %i input types, %i output types" % (len(train_data_indexed), len(dev_data_indexed), len(input_indexer), len(output_indexer)))
    if args.print_dataset:
        print("Input indexer: %s" % input_indexer)
        print("Output indexer: %s" % output_indexer)
        print("Here are some examples post tokenization and indexing:")
        for i in range(0, min(len(train_data_indexed), 10)):
            print(train_data_indexed[i])
    if args.do_nearest_neighbor:
        decoder = NearestNeighborSemanticParser(train_data_indexed)
    else:
        decoder = train_model_encdec(train_data_indexed, dev_data_indexed, input_indexer, output_indexer, args)
    print("=======DEV SET=======")
    evaluate(dev_data_indexed, decoder, use_java=args.perform_java_eval)
    print("=======FINAL PRINTING ON BLIND TEST=======")
    evaluate(test_data_indexed, decoder, print_output=False, outfile=args.test_output_path, use_java=args.perform_java_eval)



In [7]:
# Initalize arguments
args = _parse_args()
random.seed(args.seed)
np.random.seed(args.seed)

# Set arguments here
args.do_nearest_neighbor = True 

main(args)

Loaded 480 exs from file data/geo_train.tsv
Loaded 120 exs from file data/geo_dev.tsv
Loaded 280 exs from file data/geo_test.tsv
480 train exs, 120 dev exs, 238 input types, 153 output types
=======DEV SET=======


Loading JAR files: evaluator.jar


Example 49
  x      = "what is the longest river flowing through new york ?"
  y_tok  = "['_answer', '(', 'NV', ',', '_longest', '(', 'V0', ',', '(', '_river', '(', 'V0', ')', ',', '_traverse', '(', 'V0', ',', 'NV', ')', ',', '_const', '(', 'V0', ',', '_stateid', '(', "'", 'new', 'york', "'", ')', ')', ')', ')', ')']"
  y_pred = "['_answer', '(', 'NV', ',', '_longest', '(', 'V0', ',', '(', '_river', '(', 'V0', ')', ',', '_loc', '(', 'V0', ',', 'NV', ')', ',', '_const', '(', 'V0', ',', '_stateid', '(', "'", 'new', 'york', "'", ')', ')', ')', ')', ')']"
Example 99
  x      = "what is the lowest point in california ?"
  y_tok  = "['_answer', '(', 'NV', ',', '_lowest', '(', 'V0', ',', '(', '_place', '(', 'V0', ')', ',', '_loc', '(', 'V0', ',', 'NV', ')', ',', '_const', '(', 'V0', ',', '_stateid', '(', 'california', ')', ')', ')', ')', ')']"
  y_pred = "['_answer', '(', 'NV', ',', '_lowest', '(', 'V0', ',', '(', '_place', '(', 'V0', ')', ',', '_loc', '(', 'V0', ',', 'NV', ')', ',', '_const'

Loading JAR files: evaluator.jar


# Advanced Model(s)

## Seq2Seq Parser

Now you need to implement your own Seq2Seq model. There are some sections that we provided you as well to simplify things, but you should generally feel free to implement an encoder-decoder model as you wish! Specific descriptions of what each part does in this notebook can be found in the handout. A useful pytorch and Seq2Seq tutorial can also be found here:
[tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

## Embedding layer
We provided you this implementation of embedding layer using nn.module. It is very similar to what we asked you to implement in HW3 that maps words to embeddings.

In [8]:
class EmbeddingLayer(nn.Module):
    """
    Embedding layer that has a lookup table of symbols that is [full_dict_size x input_dim]. Includes dropout.
    Works for both non-batched and batched inputs
    """
    def __init__(self, input_dim: int, full_dict_size: int, embedding_dropout_rate: float):
        """
        :param input_dim: dimensionality of the word vectors
        :param full_dict_size: number of words in the vocabulary
        :param embedding_dropout_rate: dropout rate to apply
        """
        super(EmbeddingLayer, self).__init__()
        self.dropout = nn.Dropout(embedding_dropout_rate)
        self.word_embedding = nn.Embedding(full_dict_size, input_dim)

    def forward(self, input):
        """
        :param input: either a non-batched input [sent len x voc size] or a batched input
        [batch size x sent len x voc size]
        :return: embedded form of the input words (last coordinate replaced by input_dim)
        """
        embedded_words = self.word_embedding(input)
        final_embeddings = self.dropout(embedded_words)
        return final_embeddings


## Encoder
This is a RNN encoder model. This is a vetted implementation of an LSTM that is provided for your convenience; however, if you want to use a different encoder or build something yourself, you should feel free! Note that this implementation does not use GloVe embeddings, but you're free to use them if you want; you just need to modify the embedding layer. Note that updating embeddings during training is very important for good performance.

In [9]:
class RNNEncoder(nn.Module):
    """
    One-layer RNN encoder for batched inputs -- handles multiple sentences at once. To use in non-batched mode, call it
    with a leading dimension of 1 (i.e., use batch size 1)
    """
    def __init__(self, input_emb_dim: int, hidden_size: int, bidirect: bool):
        """
        :param input_emb_dim: size of word embeddings output by embedding layer
        :param hidden_size: hidden size for the LSTM
        :param bidirect: True if bidirectional, false otherwise
        """
        super(RNNEncoder, self).__init__()
        self.bidirect = bidirect
        self.hidden_size = hidden_size
        self.reduce_h_W = nn.Linear(hidden_size * 2, hidden_size, bias=True)
        self.reduce_c_W = nn.Linear(hidden_size * 2, hidden_size, bias=True)
        self.rnn = nn.LSTM(input_emb_dim, hidden_size, num_layers=1, batch_first=True,
                               dropout=0., bidirectional=self.bidirect)

    def get_output_size(self):
        return self.hidden_size * 2 if self.bidirect else self.hidden_size

    def sent_lens_to_mask(self, lens, max_length):
        """
        Converts sentence lengths to a mask tensor.

        This function creates a binary mask for a batch of sequences, where 1 represents
        actual sequence elements and 0 represents padding.

        Args:
            lens (torch.Tensor): A 1D tensor containing the actual lengths of each sequence in the batch.
            max_length (int): The maximum length to which all sequences are padded.

        Returns:
            torch.Tensor: A 2D binary mask tensor of shape (batch_size, max_length).
                        1 indicates actual sequence elements, 0 indicates padding.

        Example:
            If lens = [2, 4, 3] and max_length = 5, the output will be:
            [[1, 1, 0, 0, 0],
            [1, 1, 1, 1, 0],
            [1, 1, 1, 0, 0]]
        """
        return torch.from_numpy(np.asarray([[1 if j < lens.data[i].item() else 0 for j in range(0, max_length)] for i in range(0, lens.shape[0])]))

    def forward(self, embedded_words, input_lens):
        """
        Runs the forward pass of the LSTM
        :param embedded_words: [batch size x sent len x input dim] tensor
        :param input_lens: [batch size]-length vector containing the length of each input sentence
        :return: output (each word's representation), context_mask (a mask of 0s and 1s
        reflecting where the model's output should be considered), and h_t, a *tuple* containing
        the final states h and c from the encoder for each sentence.
        Note that output is only needed for attention, and context_mask is only used for batched attention.
        """
        # Takes the embedded sentences, "packs" them into an efficient Pytorch-internal representation
        packed_embedding = nn.utils.rnn.pack_padded_sequence(embedded_words, input_lens.cpu(), batch_first=True, enforce_sorted=False)
        # Runs the RNN over each sequence. Returns output at each position as well as the last vectors of the RNN
        # state for each sentence (first/last vectors for bidirectional)
        output, hn = self.rnn(packed_embedding)
        # Unpacks the Pytorch representation into normal tensors
        output, sent_lens = nn.utils.rnn.pad_packed_sequence(output)
        max_length = max(input_lens.data).item()
        context_mask = self.sent_lens_to_mask(sent_lens, max_length)

        if self.bidirect:
            h, c = hn[0], hn[1]
            # Grab the representations from forward and backward LSTMs
            h_, c_ = torch.cat((h[0], h[1]), dim=1), torch.cat((c[0], c[1]), dim=1)
            # Reduce them by multiplying by a weight matrix so that the hidden size sent to the decoder is the same
            # as the hidden size in the encoder
            new_h = self.reduce_h_W(h_)
            new_c = self.reduce_c_W(c_)
            h_t = (new_h, new_c)
        else:
            h, c = hn[0][0], hn[1][0]
            h_t = (h, c)
        return (output, context_mask, h_t)


## Decoder

This is the decoder part of your encoder-decoder model that you need to finish implementing. Recall that at each step of decoding, a basic decoder is given an embedded input and last hidden states from the encoder. 

Here, the *embedded_input* parameter of the forward() function is the embedded input. The parameter *state* is a tuple of (h,c) from the encoder, representing the last hidden states and the context vector. *enc_output* is the encoder outputs of each word that you may need for attention, and *enc_context_mask* is the parameter you may need if you want to achieve batch attention.




In [10]:
class RNNDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, encoder_output_size, output_dict_size, attention=False, attention_type='additive', rnn_type='lstm'):
        
        super(RNNDecoder, self).__init__()
        self.n_layers = 1
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.encoder_output_size = encoder_output_size
        self.attention = attention
        self.attention_type = attention_type
        self.rnn_type = rnn_type
        
        ### BEGIN OF FILL-IN ###
        
        if rnn_type == 'lstm':
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers=self.n_layers, batch_first=True)
        elif rnn_type == 'gru':
            self.rnn = nn.GRU(input_size, hidden_size, num_layers=self.n_layers, batch_first=True)
        else:
            raise ValueError("Unsupported RNN type")

        #TODO: implement attention
 
        self.out = nn.Linear(hidden_size, output_dict_size) 

        ### END OF FILL-IN ###

    def forward(self, embedded_input, state, enc_output, enc_context_mask):
        ### BEGIN OF FILL-IN ###

        if self.rnn_type == 'lstm':
            output, hidden = self.rnn(embedded_input, state)
        else:  # GRU
            raise Exception("implement GRU")
            output, hidden = self.rnn(embedded, state[0])
        
        # Generate output distribution
        output = self.out(output)
        log_probs = F.log_softmax(output, dim=-1)


        ### End OF FILL-IN ###
        
        return log_probs, hidden


## Helper functions
Here are some helper functions that you may find useful in transforming your input and output data.

In [11]:
def make_padded_input_tensor(exs: List[Example], input_indexer: Indexer, max_len: int, reverse_input=False) -> np.ndarray:
    """
    Takes the given Examples and their input indexer and turns them into a numpy array by padding them out to max_len.
    Optionally reverses them.
    :param exs: examples to tensor-ify
    :param input_indexer: Indexer over input symbols; needed to get the index of the pad symbol
    :param max_len: max input len to use (pad/truncate to this length)
    :param reverse_input: True if we should reverse the inputs (useful if doing a unidirectional LSTM encoder)
    :return: A [num example, max_len]-size array of indices of the input tokens
    
    Example:
    Let's say we have:
    - exs = [Example1, Example2, Example3]
    - Example1.x_indexed = [1, 2, 3, 4]
    - Example2.x_indexed = [5, 6]
    - Example3.x_indexed = [7, 8, 9]
    - input_indexer.index_of(PAD_SYMBOL) = 0
    - max_len = 5
    - reverse_input = False

    The function will return:
    np.array([
        [1, 2, 3, 4, 0],
        [5, 6, 0, 0, 0],
        [7, 8, 9, 0, 0]
    ])
    """
    if reverse_input:
        return np.array(
            [[ex.x_indexed[len(ex.x_indexed) - 1 - i] if i < len(ex.x_indexed) else input_indexer.index_of(PAD_SYMBOL)
              for i in range(0, max_len)]
             for ex in exs])
    else:
        return np.array([[ex.x_indexed[i] if i < len(ex.x_indexed) else input_indexer.index_of(PAD_SYMBOL)
                          for i in range(0, max_len)]
                         for ex in exs])

def make_padded_output_tensor(exs, output_indexer, max_len):
    """
    Similar to make_padded_input_tensor, but does it on the outputs without the option to reverse input
    :param exs:
    :param output_indexer:
    :param max_len:
    :return: A [num example, max_len]-size array of indices of the output tokens
    """
    return np.array([[ex.y_indexed[i] if i < len(ex.y_indexed) else output_indexer.index_of(PAD_SYMBOL) for i in range(0, max_len)] for ex in exs])

## Seq2Seq Parser
This is the Seq2Seq module that integrates your encoder and decoder to make a full run. In your forward() function, you should pass the input to your encoder first, then pass the output of your encoder as well as your model input to your decoder, and compute and return the loss.

You also need to implement the decode() function that takes input of your test data and return the predicted outputs. **Note that you should not need to call this function in your forward(), but you still need to implement it so that you can evaluate the result.** The evaluate() function from data.py will implicitly call it and evaluate using it.

In [19]:
###################################################################################################################
# Please fill in the code in all the specified spaces
###################################################################################################################
import math
class Seq2SeqSemanticParser(nn.Module):
    def __init__(self, input_indexer, output_indexer, emb_dim, hidden_size, embedding_dropout=0.2,
                 bidirect=True, use_luong=True, attention=True, output_max_len=65, reverse_input=False):
        # We've included some args for setting up the input embedding and encoder
        # You'll need to add code for output embedding and decoder
        super(Seq2SeqSemanticParser, self).__init__()
        self.input_indexer = input_indexer
        self.output_indexer = output_indexer
        
        self.input_emb = EmbeddingLayer(emb_dim, len(input_indexer), embedding_dropout)
        self.encoder = RNNEncoder(input_emb_dim=emb_dim, hidden_size=hidden_size, bidirect=bidirect)

        ### BEGIN OF FILL-IN ###
        self.output_emb = EmbeddingLayer(emb_dim, len(output_indexer), embedding_dropout)
        encoder_output_size = self.encoder.get_output_size()
        self.decoder = RNNDecoder(input_size=emb_dim, hidden_size=hidden_size, 
                                    encoder_output_size=encoder_output_size, output_dict_size=len(output_indexer),
                                    attention=attention)
        self.output_max_len = output_max_len
       
        ### END OF FILL-IN ###
    
    def forward(self, x_tensor, inp_lens_tensor, y_tensor, out_lens_tensor, args):
        """
        :param x_tensor/y_tensor: either a non-batched input/output [sent len] vector of indices or a batched input/output
        [batch size x sent len]. y_tensor contains the gold sequence(s) used for training
        :param inp_lens_tensor/out_lens_tensor: either a vector of input/output length [batch size] or a single integer.
        lengths aren't needed if you don't batchify the training.
        :return: loss of the batch
        """
        ### BEGIN OF FILL-IN ###
        batch_size, max_out_len = y_tensor.shape

        # Encode input
        x_embedded = self.input_emb(x_tensor)
        enc_output_each_word, enc_context_mask, enc_final_states = self.encode_input(x_embedded, inp_lens_tensor)

        # Prepare decoder input (add start token)
        start_token = torch.full((batch_size, 1), SOS_SYMBOL, device=y_tensor.device)
        y_input = torch.cat([start_token, y_tensor[:, :-1]], dim=1)  # Teacher forcing
        y_embedded = self.output_emb(y_input)

        # Initialize decoder state
        dec_hidden = enc_final_states

        # Decode
        # batch size x seq length x output dict size
        log_probs, _ = self.decoder.forward(y_embedded, enc_final_states, enc_output_each_word, enc_context_mask)
        
        # Reshape log_probs for NLLLoss becomes [(batch size x seq len) x output dict size]
        log_probs_reshaped = log_probs.view(-1, log_probs.size(-1))
        # y_tensor is [batch size x seq len] -> [(batch size x seq len) x 1]
        y_tensor_reshaped = y_tensor.view(-1)

        # Calculate loss
        loss_function = nn.NLLLoss(ignore_index=self.pad_token_idx)
        loss = loss_function(log_probs_reshaped, y_tensor_reshaped)

        return loss
        
        ### END OF FILL-IN ###

    def decode(self, test_data: List[Example]) -> List[List[Derivation]]:

        """
        :param test_data: input test data as List[Example]. You can find the definition of the class Example() in data.py
        
        :return: your predicted outputs as List[List[Derivation]]. You can find more descriptions of the class Derivation() in data.py
        """

        ### BEGIN OF FILL-IN ###
        all_derivations = []
        beam_size = 5  # You can adjust this

        for example in test_data:
            x_tensor = torch.tensor([example.x_indexed])
            inp_lens_tensor = torch.tensor([len(example.x_indexed)])
            
            enc_output_each_word, enc_context_mask, enc_final_states = self.encode_input(x_tensor, inp_lens_tensor)
            
            # Initialize beam with start symbol
            init_seq = [SOS_SYMBOL]
            beam = Beam(beam_size)
            beam.add((init_seq, enc_final_states), 0)  # Initial log prob is 0
            
            completed_sequences = []
            
            for _ in range(self.output_max_len):
                new_beam = Beam(beam_size)
                
                for seq, state in beam.get_elts():
                    last_token_idx = self.output_indexer.index_of(seq[-1])
                    last_token = torch.tensor([[last_token_idx]])
                    decoder_input_embedded = self.output_emb.forward(last_token)
                    log_probs, new_state = self.decoder.forward(decoder_input_embedded, state, enc_output_each_word, enc_context_mask)

                    log_probs = log_probs.squeeze(0) 
                    top_log_probs, top_indices = log_probs.topk(beam_size)
                
                    for log_prob, idx in zip(top_log_probs, top_indices):
                        token = self.output_indexer.get_object(idx.item())
                        new_seq = seq + [token]
                        new_score = beam.scores[beam.elts.index((seq, state))] + log_prob.item()
                        new_beam.add((new_seq, new_state), new_score)
                        
                        if token == EOS_SYMBOL:
                            completed_sequences.append((new_seq, new_score))
                
                beam = new_beam
                
                # Early stopping if beam is empty (all sequences ended with EOS)
                if len(beam) == 0:
                    break
        
        for seq, state in beam.get_elts():
            score = beam.scores[beam.elts.index((seq, state))]
            completed_sequences.append((seq, score))
        
        all_sequences = sorted(completed_sequences + list(beam.get_elts_and_scores()), key=lambda x: x[1], reverse=True)
        top_sequences = all_sequences[:beam_size]

        example_derivations = []
        for seq, score in top_sequences:
            output_tokens = [token for token in seq[1:] if token not in {SOS_SYMBOL, EOS_SYMBOL}]
            derivation = Derivation(example, score, output_tokens)
            example_derivations.append(derivation)
        
        all_derivations.append(example_derivations)

        return all_derivations

        
        ### END OF FILL-IN ###


    def encode_input(self, x_tensor, inp_lens_tensor):
        """
        Runs the encoder (input embedding layer and encoder as two separate modules) on a tensor of inputs x_tensor with
        inp_lens_tensor lengths.
        YOU DO NOT HAVE TO USE THIS FUNCTION. It's meant to illustrate the usage of EmbeddingLayer and RNNEncoder
        as they're given to you, as well as show what kinds of inputs/outputs you need from your encoding phase.
        :param x_tensor: [batch size, sent len] tensor of input token indices
        :param inp_lens_tensor: [batch size] vector containing the length of each sentence in the batch
        :param model_input_emb: EmbeddingLayer
        :param model_enc: RNNEncoder
        :return: the encoder outputs (per word), the encoder context mask (matrix of 1s and 0s reflecting which words
        are real and which ones are pad tokens), and the encoder final states (h and c tuple). ONLY THE ENCODER FINAL
        STATES are needed for the basic seq2seq model. enc_output_each_word is needed for attention, and
        enc_context_mask is needed to batch attention.

        E.g., calling this with x_tensor (0 is pad token):
        [[12, 25, 0],
        [1, 2, 3],
        [2, 0, 0]]
        inp_lens = [2, 3, 1]
        will return outputs with the following shape:
        enc_output_each_word = 3 x 3 x dim, enc_context_mask = [[1, 1, 0], [1, 1, 1], [1, 0, 0]],
        enc_final_states = 3 x dim
        """
        input_emb = self.input_emb.forward(x_tensor)
        (enc_output_each_word, enc_context_mask, enc_final_states) = self.encoder.forward(input_emb, inp_lens_tensor)
        enc_final_states_reshaped = (enc_final_states[0].unsqueeze(0), enc_final_states[1].unsqueeze(0))
        return (enc_output_each_word, enc_context_mask, enc_final_states_reshaped)





# Train

This is your main training function. You may want to instantiate a Seq2SeqSemanticParser() model and an optimizer, train it over epochs, and evaluate on your dev_data after training.

In [20]:
def train_model_encdec(train_data: List[Example], dev_data: List[Example], input_indexer, output_indexer, args) -> Seq2SeqSemanticParser:
    """
    Function to train the encoder-decoder model on the given data.
    :param train_data:
    :param dev_data: Development set in case you wish to evaluate during training
    :param input_indexer: Indexer of input symbols
    :param output_indexer: Indexer of output symbols
    :param args:
    :return:
    """
    
    # Create indexed input
    input_max_len = np.max(np.asarray([len(ex.x_indexed) for ex in train_data]))
    all_train_input_data = make_padded_input_tensor(train_data, input_indexer, input_max_len, reverse_input=args.reverse_input)
    all_test_input_data = make_padded_input_tensor(dev_data, input_indexer, input_max_len, reverse_input=args.reverse_input)

    output_max_len = np.max(np.asarray([len(ex.y_indexed) for ex in train_data]))
    all_train_output_data = make_padded_output_tensor(train_data, output_indexer, output_max_len)
    all_test_output_data = make_padded_output_tensor(dev_data, output_indexer, output_max_len)

    if args.print_dataset:
        print("Train length: %i" % input_max_len)
        print("Train output length: %i" % np.max(np.asarray([len(ex.y_indexed) for ex in train_data])))
        print("Train matrix: %s; shape = %s" % (all_train_input_data, all_train_input_data.shape))

    # First create a model. Then loop over epochs, loop over examples, and given some indexed words
    # call your seq-to-seq model, accumulate losses, update parameters, 
    # Some started code is provided below, but you are NOT necessary to follow them
    
    ### BEGIN OF FILL-IN ###
    model = Seq2SeqSemanticParser(input_indexer, output_indexer, args.emb_dim, args.hidden_size)
    model.to(args.device)

    # Define optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)


    for epoch in tqdm(range(args.epochs)):
        model.train()
        total_loss = 0
        batch_order = list(range(0, int(all_train_input_data.shape[0] / args.batch_size)))
        random.shuffle(batch_order)

        for batch_idx in batch_order:
            start_idx = batch_idx * args.batch_size
            end_idx = (batch_idx + 1) * args.batch_size

            x_tensor = torch.from_numpy(all_train_input_data[start_idx:end_idx]).to(args.device)
            y_tensor = torch.from_numpy(all_train_output_data[start_idx:end_idx]).to(args.device)
            inp_lens_tensor = torch.sum(x_tensor != 0, dim=1)
            out_lens_tensor = torch.sum(y_tensor != 0, dim=1)

            optimizer.zero_grad()

            loss = model(x_tensor, inp_lens_tensor, y_tensor, out_lens_tensor, args)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(batch_order)
        print(f"Epoch {epoch+1}, Average Loss: {avg_loss:.4f}")

    
    ### END OF FILL-IN ###

    return model

# Now run your code!

Now that you finished implementing your model, you can run your code just as you did for the nearest neighbor baseline! Make sure to include any arguments you would like in the *add_models_args()* function and set *do_nearest_neighbor=False*. Note that it takes roughly 8 minutes if you are running on CPU and 2 minutes if you are running on GPU. Besides, don't forget to use the *evaluate()* function to generate and save outputs for your test data and submit them to gradescope! You can do it in your *main()* function as well.

In [21]:
# Initalize arguments
args = _parse_args()
random.seed(args.seed)
np.random.seed(args.seed)

# Set arguments here
args.do_nearest_neighbor = False 

main(args)

Loaded 480 exs from file data/geo_train.tsv
Loaded 120 exs from file data/geo_dev.tsv
Loaded 280 exs from file data/geo_test.tsv
480 train exs, 120 dev exs, 238 input types, 153 output types


  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

# Submission

These are the files you have to submit to gradescope

1. Report
2. The prediction output of basic encoder-decoder and attention model as "test_output.tsv" (from the Files section of sidebar)